In [37]:
# importations sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate, cross_val_score, validation_curve, StratifiedKFold
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [38]:
# importations pytorch
import torch
from torch.utils.data import TensorDataset                    # to build data set with pytorch

In [39]:
dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


C:\Users\jomue\AppData\Local\Temp\ipykernel_8480\918415353.py:1: DtypeWarning: Columns (36,38,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset=pd.read_csv("https://raw.githubusercontent.com/Jomuel-net/AI-finance-project/refs/heads/main/Data%20set/data_bank_alaska.csv")


In [40]:
head=dataset.head()
info=dataset.info()
print(head)
print(info)                                                                         # 28632 rows and 78 columns, there are 3 Dtypes ( float, int and object=[string] )
target_name= "action_taken_name"                                                         # purpose name
data , target = dataset.drop(columns=target_name), dataset[target_name]             # data to browse an target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 78 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   as_of_year                      28632 non-null  int64  
 1   respondent_id                   28632 non-null  object 
 2   agency_name                     28632 non-null  object 
 3   agency_abbr                     28632 non-null  object 
 4   agency_code                     28632 non-null  int64  
 5   loan_type_name                  28632 non-null  object 
 6   loan_type                       28632 non-null  int64  
 7   property_type_name              28632 non-null  object 
 8   property_type                   28632 non-null  int64  
 9   loan_purpose_name               28632 non-null  object 
 10  loan_purpose                    28632 non-null  int64  
 11  owner_occupancy_name            28632 non-null  object 
 12  owner_occupancy                 

In [41]:
# valeurs numeriques
dataset.select_dtypes(include='number')                      # on a 45 colonnes à données numériques 

,as_of_year,agency_code,loan_type,property_type,loan_purpose,owner_occupancy,loan_amount_000s,preapproval,action_taken,msamd,...,edit_status_name,edit_status,sequence_number,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units,application_date_indicator
0,2017,9,3,1,1,1,359,3,1,11260.0,...,NaN,NaN,NaN,6702.0,17.170000,89400.0,94.010002,1651.0,2219.0,NaN
1,2017,5,1,1,1,1,99,2,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,9,2,1,1,1,247,3,1,21820.0,...,NaN,NaN,NaN,11450.0,20.030001,84800.0,99.690002,2923.0,3992.0,NaN
3,2017,5,1,1,1,1,292,3,1,11260.0,...,NaN,NaN,NaN,3864.0,22.490000,89400.0,71.989998,979.0,1328.0,NaN
4,2017,3,1,1,1,1,589,3,1,11260.0,...,NaN,NaN,NaN,1941.0,56.880001,89400.0,58.340000,179.0,318.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28627,2017,7,1,1,3,1,341,3,1,11260.0,...,NaN,NaN,NaN,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0,NaN
28628,2017,7,1,1,3,2,80,3,3,11260.0,...,NaN,NaN,NaN,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0,NaN
28629,2017,7,3,1,3,1,199,3,1,NaN,...,NaN,NaN,NaN,4619.0,71.339996,78400.0,130.669998,213.0,680.0,NaN
28630,2017,9,1,1,1,1,280,3,1,NaN,...,NaN,NaN,NaN,4619.0,71.339996,78400.0,130.669998,213.0,680.0,NaN


In [42]:
#==qualité des données==

dataset.isnull().sum()                                                                              # nombre de valeurs manquantes par colonnes 

# retirer les colonnes qui ont plus de 50% de valeurs manquantes

pourcentage_manquant=dataset.isnull().mean()*100                                                    # pourcentage manquant par colonne                                               

colonnes_a_supprimer=pourcentage_manquant[pourcentage_manquant>50].index.tolist()                   # on retire les colonnes qui ont plus de 50% de valeurs manquantes

data_clean=data.drop(columns=colonnes_a_supprimer)                                                  # nouvelles données


# doublons
data_clean.duplicated().sum()                                                                       # nombre de doublons par lignes

data_clean.drop_duplicates()                                                                        # retire les lignes oublons




,as_of_year,respondent_id,agency_name,agency_abbr,agency_code,loan_type_name,loan_type,property_type_name,property_type,loan_purpose_name,...,hoepa_status_name,hoepa_status,lien_status_name,lien_status,population,minority_population,hud_median_family_income,tract_to_msamd_income,number_of_owner_occupied_units,number_of_1_to_4_family_units
0,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,VA-guaranteed,3,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,6702.0,17.170000,89400.0,94.010002,1651.0,2219.0
1,2017,0000005892,National Credit Union Administration,NCUA,5,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,NaN,NaN,NaN,NaN,NaN,NaN
2,2017,0000451965,Consumer Financial Protection Bureau,CFPB,9,FHA-insured,2,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,11450.0,20.030001,84800.0,99.690002,2923.0,3992.0
3,2017,0000005912,National Credit Union Administration,NCUA,5,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,3864.0,22.490000,89400.0,71.989998,979.0,1328.0
4,2017,92-0164378,Federal Deposit Insurance Corporation,FDIC,3,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,1941.0,56.880001,89400.0,58.340000,179.0,318.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28627,2017,7197000003,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,Not a HOEPA loan,2,Secured by a first lien,1,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0
28628,2017,04-3660901,Department of Housing and Urban Development,HUD,7,Conventional,1,One-to-four family dwelling (other than manufa...,1,Refinancing,...,Not a HOEPA loan,2,Secured by a first lien,1,6017.0,22.520000,89400.0,119.639999,1470.0,1908.0
28629,2017,7197000003,Department of Housing and Urban Development,HUD,7,VA-guaranteed,3,One-to-four family dwelling (other than manufa...,1,Refinancing,...,Not a HOEPA loan,2,Secured by a first lien,1,4619.0,71.339996,78400.0,130.669998,213.0,680.0
28630,2017,0000280110,Consumer Financial Protection Bureau,CFPB,9,Conventional,1,One-to-four family dwelling (other than manufa...,1,Home purchase,...,Not a HOEPA loan,2,Secured by a first lien,1,4619.0,71.339996,78400.0,130.669998,213.0,680.0


In [43]:
# suite vérification de la qualité des colonnes
col="as_of_year"                                # non pertinente
print(data_clean[col].unique())                 # affiche toutes les valeurs distinctes de cette colonne

print(data_clean[col].nunique())                # affiche le nombre de valeurs distinctes de cette colonne

# suppression de la colonne col
data_clean=data_clean.drop(columns=col)

# suppression de la colonne agency_abbr et agency_code
data_clean=data_clean.drop(columns=["agency_abbr","agency_code","state_name", "state_abbr", "state_code"])

# suppression donc de cette colonne
data_clean=data_clean.drop(columns="preapproval")

# plus genralement, identification des doublons categorique/numerique et suppression des equivalents catégoriques


# Identifier les colonnes qui finissent par "_name"
name_cols = [col for col in data_clean.columns if col.endswith("_name")]

# Extraire les radicaux
radicals = [col[:-5] for col in name_cols]   # enlève "_name"

# Supprimer les colonnes qui sont exactement un radical
cols_to_drop = [col for col in data_clean.columns if col in radicals]

# Garder toutes les autres
columns_to_keep = [col for col in data_clean.columns if col not in cols_to_drop]

data_clean=data_clean.drop(columns=cols_to_drop)

[2017]
1


In [44]:
data_clean.info()                                                                                    # finalement seules 32 colonnnes sont utiles dans le data set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28632 entries, 0 to 28631
Data columns (total 31 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   respondent_id                   28632 non-null  object 
 1   agency_name                     28632 non-null  object 
 2   loan_type_name                  28632 non-null  object 
 3   property_type_name              28632 non-null  object 
 4   loan_purpose_name               28632 non-null  object 
 5   owner_occupancy_name            28632 non-null  object 
 6   loan_amount_000s                28632 non-null  int64  
 7   preapproval_name                28632 non-null  object 
 8   action_taken                    28632 non-null  int64  
 9   msamd_name                      22213 non-null  object 
 10  county_name                     28514 non-null  object 
 11  county_code                     28523 non-null  float64
 12  census_tract_number             

In [ ]:
# types des données
numerical_features=data_clean.select_dtypes(include='number').columns.tolist()                         # stockage des données numériques (13 colonnes)
#penser à requestionner les colonnes numeriques importantes
categorical_features=list(set(data_clean.columns)-set(numerical_features))                             # stockage des données catégoriques(18 colonnes)
print(len(numerical_features))
print(len(categorical_features))

13
18


In [46]:
# completion des nan
data_clean[numerical_features] = data_clean[numerical_features].fillna(data_clean[numerical_features].mean())                         # remplacer les valeurs manquanntes par la moyenne de la colonne considérée

data_clean[categorical_features] = data_clean[categorical_features].apply(lambda col: col.fillna(col.mode()[0]))                      # remplace par la plus occurente (mode)


In [47]:
# implementation du modele hgbc avec les deux types de donnes

# identification des valeurs catégoriques
categorical_features= list(set(data_clean.columns)-set(numerical_features))

#implemetation fusionnée

# Let's define a predictive model
numeric= StandardScaler()
categoric = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric",numeric, numerical_features),
        ("categoric",categoric, categorical_features)
    ]
)
# model pipeloine and scaling complete
model_pipeline_full=Pipeline([ 
    ('preprocessor', preprocessor),
    ('rfc', RandomForestClassifier())
])

# data preparation
X_full=data_clean[numerical_features+categorical_features]
Y= target

# training
model_pipeline_full.fit(X_full,Y)

# predictive model
y_pred=model_pipeline_full.predict(X_full)

# evaluation of the model using cross validation

cv_results= cross_validate(
    model_pipeline_full,
    X_full,
    Y,
    cv=StratifiedKFold(n_splits=10),
    scoring= 'accuracy',
    return_train_score= True,
    return_estimator= True
)
# print the accuracy

mean_accuracy_full = cv_results['test_score'].mean()
print(f"Accuracy_full  (10-fold CV) : {mean_accuracy_full:.2f}")

Accuracy_full  (10-fold CV) : 0.97


In [48]:
# implementation du modele hgbc avec les deux types de donnes

# identification des valeurs catégoriques
categorical_features= list(set(data_clean.columns)-set(numerical_features))

#implemetation fusionnée

# Let's define a predictive model
numeric= StandardScaler()
categoric = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("numeric",numeric, numerical_features),
        ("categoric",categoric, categorical_features)
    ]
)
# model pipeloine and scaling complete
model_pipeline_full=Pipeline([ 
    ('preprocessor', preprocessor),
    ('adc', AdaBoostClassifier())
])

# data preparation
X_full=data_clean[numerical_features+categorical_features]
Y= target

# training
model_pipeline_full.fit(X_full,Y)

# predictive model
y_pred=model_pipeline_full.predict(X_full)

# evaluation of the model using cross validation

cv_results= cross_validate(
    model_pipeline_full,
    X_full,
    Y,
    cv=StratifiedKFold(n_splits=10),
    scoring= 'accuracy',
    return_train_score= True,
    return_estimator= True
)
# print the accuracy

mean_accuracy_full = cv_results['test_score'].mean()
print(f"Accuracy_full  (10-fold CV) : {mean_accuracy_full:.2f}")

Accuracy_full  (10-fold CV) : 0.96


In [49]:
print(pd.Series(Y).value_counts())   # occurrences


action_taken_name
Loan originated                                14430
Loan purchased by the institution               6415
Application withdrawn by applicant              3495
Application denied by financial institution     3075
File closed for incompleteness                   756
Application approved but not accepted            461
Name: count, dtype: int64


Let's Build and train a classification MLP on our data  with pytorch tools.

Step 1: Load the dataset using sklearn and create a custom PyTorch Dataset for this data.

In [ ]:
# implementation of the DNN

encoder=LabelEncoder()                                                                  # instance encoder
target_encoded=encoder.fit_transform(target)                                            # target encoding for tensor management

X_prepared=preprocessor.fit_transform(X_full)                                           # prepares X_full for pytorch( in numpy)

X=torch.tensor(X_prepared,dtype=torch.float32)                                          # data tensor building

means = X.mean(dim=0, keepdim=True)                                                     # mean
stds = X.std(dim=0, keepdim=True)                                                       # ecart type
X_standardized= (X - means) / stds                                                      # scaling

y=torch.tensor(target_encoded, dtype=torch.long)                                        # target tensor building                                           

data_set_torch=TensorDataset(X_standardized,y)                                          # data set building 

sample0, target0 = data_set_torch[0]
print (sample0.shape, target0.shape)                                                    # on obtient des vecteurs de dimensiosn 328 et le tzrget etant scalaore 

torch.Size([328]) torch.Size([])



Step 2: Create data loaders for training, validation, and testing.